In [1]:
%matplotlib inline

import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt

from biom import load_table
from biom.util import biom_open

from qiime.parse import parse_mapping_file
from qiime.format import format_mapping_file
from skbio.io.util import open_file
from scipy.stats import pearsonr, spearmanr

def load_mf(fn):
    with open_file(fn, 'U') as f:
        mapping_data, header, _ = parse_mapping_file(f)
        _mapping_file = pd.DataFrame(mapping_data, columns=header)
        _mapping_file.set_index('SampleID', inplace=True)
    return _mapping_file

def write_mf(f, _df):
    with open_file(f, 'w') as fp:
        lines = format_mapping_file(['SampleID'] + _df.columns.tolist(),
                                    list(_df.itertuples()))
        fp.write(lines+'\n')

In [2]:
ls combined-gevers-suchdolski/ibd-comparison/

mapping-file-Healthy.15000.txt      otu-table-IBD.15000.40pct.biom
mapping-file-IBD.15000.txt          otu-table-IBD.15000.biom
otu-table-Healthy.15000.40pct.biom  taxa-summaries-Healthy-40pct/
otu-table-Healthy.15000.biom        taxa-summaries-IBD-40pct/


# Filter out all but fecal samples from both studies

In [4]:
mf = load_mf('combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.txt')
bt = load_table('combined-gevers-suchdolski/otu-table.15000.biom')

mf = mf.loc[list(bt.ids())]
mf = mf[mf.BIOPSY_LOCATION.isin(['stool', 'no_data'])]

bt.filter(mf.index.tolist(), inplace=True)

with biom_open('combined-gevers-suchdolski/otu-table.15000.fecal-only.hdf5.biom', 'w') as f:
    bt.to_hdf5(f, 'Notebook filtering data')
    
with open('combined-gevers-suchdolski/otu-table.15000.fecal-only.json.biom', 'w') as f:
    f.write(bt.to_json('Notebook filtering data'))

write_mf('combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.txt', mf)

In [5]:
!beta_diversity_through_plots.py \
-i combined-gevers-suchdolski/otu-table.15000.fecal-only.hdf5.biom \
-m combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.txt \
-o combined-gevers-suchdolski/beta-fecal-only \
-t gg_13_8_otus/trees/97_otus.tree

/Users/yoshikivazquezbaeza/.virtualenvs/qiime-191/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -1.4156344708 and the largest is 36.1591245361.
  RuntimeWarning
/Users/yoshikivazquezbaeza/.virtualenvs/qiime-191/lib/python2.7/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:107: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes 

Redo some plots to fit add a column.

In [1]:
!make_emperor.py \
-i combined-gevers-suchdolski/beta-fecal-only/unweighted_unifrac_pc.txt \
-m combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.combined-columns.txt \
-o combined-gevers-suchdolski/beta-fecal-only/unweighted_unifrac_emperor_pcoa_plot/

!make_emperor.py \
-i combined-gevers-suchdolski/beta-fecal-only/weighted_unifrac_pc.txt \
-m combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.combined-columns.txt \
-o combined-gevers-suchdolski/beta-fecal-only/weighted_unifrac_emperor_pcoa_plot/